# hw 10
### due May 4, 12:30 pm

### data: `affairs_samp`
We will work again with survey data on marital infidelity from a Psychology study from the 1960s. 

This time, I have randomly generated two samples of equally small size, identified by the `sampid` variable.

Get this data

#### more info
Each observation represents one respondent's answers.

**See here for more information** on this dataset (same as last time)
https://vincentarelbundock.github.io/Rdatasets/doc/AER/Affairs.html

### goal: let's do some p-hacking

We're going to do some minor p-hacking to explore a bit more why it's bad. See the links in the `hypotest_three` slides for more detailed analyses of ways p-hacking has been abused.

### code used to generate samples

# Q1
Load the data as usual. **For `sampid = 0` (see data explanation)**

- **create a pivot table** (or grouped summary information in pandas, same as last time) with 
    - group variables `young` and `children`
    - summary statistics for `some_affairs`: `mhat` (sample mean), `shat` (sample s.d.), `n` number of samples per group, `sehat` (s.d. of the mean using CLT, aka standard error)

- **create another pivot table** (or grouped summary information in pandas, same as last time) with 
    - group variables `young` and `gender`
    - same summary stats for `some_affairs`
    
**Note for excel users**: You don't need to rename the pivot table columns to those listed above.

Pandas users also can call things what they want, so long as the names are clear.

In [1]:
import pandas as pd
import scipy.stats as stats

In [2]:
d = pd.read_csv('https://raw.githubusercontent.com/brendanrbrown/stor155_sp21/main/data/affairs_samp.csv')

#### some helpful functions
Many ways to do this. Here I balance good programming practices with the goal of not introducing too many new things.

In [3]:
def sample_stats(data, sub_gp):
    group = ['young', 'sampid'] + [sub_gp]
    data = data.loc[:, group + ['some_affairs']].groupby(group).agg(['mean', 'std', 'size']
                                  ).droplevel(0, axis = 1).rename(columns = {'mean': 'mhat', 'std': 'shat', 'size': 'n'})
    
    return data.assign(sehat = lambda x: x.shat / x.n**.5).reset_index(level = sub_gp)

Actually we'll do Q1, 2 all in one go, but to satisfy this question let's just print the results.


**Note there is some freaky stuff going on here**: A number of the values are the same. This is by chance! It was made more likely by the fact that 1) this is a small sample, 2) I constrained all subgroup sample sizes to be the same.

Intuitively though, it's not great, and it gives you a sense yet a gain that small sample sizes can lead to bad or weird results.

In [4]:
sample_stats(d, 'children')

children    mhat      shat   n     sehat
young sampid                                         
False 0            no  0.3750  0.500000  16  0.125000
      0           yes  0.1875  0.403113  16  0.100778
      1            no  0.3750  0.500000  16  0.125000
      1           yes  0.1875  0.403113  16  0.100778
True  0            no  0.2500  0.447214  16  0.111803
      0           yes  0.4375  0.512348  16  0.128087
      1            no  0.1875  0.403113  16  0.100778
      1           yes  0.4375  0.512348  16  0.128087

In [5]:
sample_stats(d, 'gender')

gender    mhat      shat   n     sehat
young sampid                                        
False 0       female  0.3125  0.478714  16  0.119678
      0         male  0.2500  0.447214  16  0.111803
      1       female  0.3750  0.500000  16  0.125000
      1         male  0.1875  0.403113  16  0.100778
True  0       female  0.3750  0.500000  16  0.125000
      0         male  0.3125  0.478714  16  0.119678
      1       female  0.4375  0.512348  16  0.128087
      1         male  0.1875  0.403113  16  0.100778

### getting data from the output

The two data frames above give all we need. Just use .loc to get what you want for the next sections.

The only tricky part here is that the index (row labels) now has two components, True/False and 0/1.

To avoid dealing with that I will just reset the index, so we can work with .loc statements that should be more familiar to you.

In [6]:
children_stats = sample_stats(d, 'children').reset_index()
children_stats

,young,sampid,children,mhat,shat,n,sehat
0,False,0,no,0.3750,0.500000,16,0.125000
1,False,0,yes,0.1875,0.403113,16,0.100778
2,False,1,no,0.3750,0.500000,16,0.125000
3,False,1,yes,0.1875,0.403113,16,0.100778
4,True,0,no,0.2500,0.447214,16,0.111803
5,True,0,yes,0.4375,0.512348,16,0.128087
6,True,1,no,0.1875,0.403113,16,0.100778
7,True,1,yes,0.4375,0.512348,16,0.128087


In [7]:
gender_stats = sample_stats(d, 'gender').reset_index()
gender_stats

,young,sampid,gender,mhat,shat,n,sehat
0,False,0,female,0.3125,0.478714,16,0.119678
1,False,0,male,0.2500,0.447214,16,0.111803
2,False,1,female,0.3750,0.500000,16,0.125000
3,False,1,male,0.1875,0.403113,16,0.100778
4,True,0,female,0.3750,0.500000,16,0.125000
5,True,0,male,0.3125,0.478714,16,0.119678
6,True,1,female,0.4375,0.512348,16,0.128087
7,True,1,male,0.1875,0.403113,16,0.100778


Now to get, for example, the data needed for a hypothesis test between male/female for `young == False` and `sampid == 0` use .loc as we have before.

Remember pandas series require us to use ~ for 'not', | for 'or' and & for 'and'

In [8]:
# gender_stats.young is a shortcut for gender_stats.loc[:, "young"]
# if selecting rows, you can use .loc[your conditions] as a shortcut for .loc[your conditions, :]

gender_stats.loc[gender_stats.loc[:, "young"].eq(False) & gender_stats.loc[:, "sampid"].eq(0)]

,young,sampid,gender,mhat,shat,n,sehat
0,False,0,female,0.3125,0.478714,16,0.119678
1,False,0,male,0.2500,0.447214,16,0.111803


In [9]:
# childre_stats now

children_stats.loc[children_stats.loc[:, "young"].eq(False) & children_stats.loc[:, "sampid"].eq(0)]

,young,sampid,children,mhat,shat,n,sehat
0,False,0,no,0.3750,0.500000,16,0.125000
1,False,0,yes,0.1875,0.403113,16,0.100778


# Q2

Using data from the pivot tables above:

- for **young = FALSE**,
    - **calculate** the p-values for a difference of means test between the no-children and children subgroups
    - **calculate** the p-values for a difference of means test between the female and male gender groups
    
- for **young = TRUE**,
    - **calculate** the p-values for a difference of means test between the no-children and children subgroups
    - **calculate** the p-values for a difference of means test between the female and male gender groups
    
In each case you will do a **two-sided** hypothesis test, meaning the alternative hypothesis is that the true means (proportions) are not equal.


### ANSWER for Q2, 4
actually we're going to do all of the p-value questions together here

In [10]:
def p_vals(data):
    z = -abs(data.mhat.iloc[0] - data.mhat.iloc[1]) / (data.sehat.iloc[0]**2 + data.sehat.iloc[1]**2)**.5
    p = 2*stats.norm.cdf(z)
    return p

In [11]:
# creatingpairs of young/sampid
ps = pd.DataFrame([[s, b] for s in [0, 1] for b in [True, False]], columns = ["sampid", "young"])
ps

,sampid,young
0,0,True
1,0,False
2,1,True
3,1,False


#### gender comparisons

In [12]:
# let's go one-by one first

# compare the genders within not young and in sampid 0
# you'll see why I used eq on the young column below, though that looks dumb here
p_vals(gender_stats.loc[gender_stats.loc[:, "young"].eq(True) & gender_stats.loc[:, "sampid"].eq(0)])

0.7179816667337766

In [13]:
# young and in sampid 0
p_vals(gender_stats.loc[gender_stats.loc[:, "young"].eq(False) & gender_stats.loc[:, "sampid"].eq(0)])

0.702745926377029

In [14]:
# all together now with apply
ps_gender = ps.apply(lambda d: p_vals(gender_stats.loc[gender_stats.loc[:, "young"].eq(d.young) & gender_stats.loc[:, "sampid"].eq(d.sampid)]),
               axis = 1)

ps_gender

0    0.717982
1    0.702746
2    0.125047
3    0.242908
dtype: float64

In [15]:
ps = ps.assign(ps_gender = ps_gender)
ps

,sampid,young,ps_gender
0,0,True,0.717982
1,0,False,0.702746
2,1,True,0.125047
3,1,False,0.242908


#### children comparisons

In [16]:
# same thing but with children_stats instead of gender_stats

ps_children = ps.apply(lambda d: p_vals(children_stats.loc[gender_stats.loc[:, "young"].eq(d.young) & children_stats.loc[:, "sampid"].eq(d.sampid)]),
               axis = 1)

ps_children

0    0.270105
1    0.242908
2    0.125047
3    0.242908
dtype: float64

In [17]:
ps = ps.assign(ps_children = ps_children)
ps

,sampid,young,ps_gender,ps_children
0,0,True,0.717982,0.270105
1,0,False,0.702746,0.242908
2,1,True,0.125047,0.125047
3,1,False,0.242908,0.242908


# Q3
Using the p-values from Q2, 

- **state** in which cases you can reject the null at a level `alpha = 0.1`
- **state** which case gives smallest p-value obtained


#### A: 
Remember we're just looking at `sampid == 0` here. You cannot reject in any case.

# Q4

**Repeat Q1** but for the sample `sampid = 1`.

#### A: see above

# Q5

**Repeat Q2** but for the data created in Q4.

#### A: see above

# Q6

**Repeat Q3** but for the p-values from Q4.


#### A: 
You still cannot reject. However, you almost can for the young group.

# Q7

Compare your answers in Q5, Q3.

- **State** using concepts from class whether or not this provides strong evidence for your conclusions in Qs 3, 6.

- If your conclusions changed: Give one reason (from class concepts) why this might be reasonable.

- If your conclusions did not change: Give one reason (from class concepts) why they might differ.

**A:** In the case of `sample = 0` you can't reject the null in either gender comparison, among the young or not young groups. But in the `sample = 1` case you can *almost* reject in both gender and children comparisons among the young group. If you were willing to accept a slightly larger alpha, then you would have successfully p-hacked the dataset and found a 'significant' result!

These differences can show up just by random chance because of sampling --- especially when the sample sizes are small, as they are here. In the second sample you are just going out in search of some, any, comparison to make that will lead to a 'significant' enough hypothesis test. But that result is not replicatable, as the results for the first sample show.